# Data Preprocessing

## Librerie utili

In [1]:
import json
import pandas as pd
from pymongo import MongoClient
import medspacy

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giuse\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giuse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giuse\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Importo le secrets del progetto

In [ ]:
with open("secrets.json") as f:
    secrets = json.load(f)
    mongo_string = secrets["mongo_string"]

## Connessione al DataLake su MongoDB

In [2]:
client = MongoClient(mongo_string)

db = client.noteevents
noteevents_db = db.noteevents_db

## Preprocessing

### Definisco la funzione di Eliminazione delle stopwords e Lemmatizing

In [3]:
import re
from nltk.corpus import stopwords

def tokenize_and_lemmatize(text):
    # Tokenizzazione delle parole
    tokens = word_tokenize(text)
    
    # Inizializzazione del lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Rimozione delle stopwords e simboli
    stop_words = set(stopwords.words('english'))
    symbols = ['[', '*', '+']
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words and not re.match(r'^\W+$', token) and not any(symbol in token for symbol in symbols)]
    
    # Lemmatizzazione delle parole
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    
    # Riaccorpamento del testo lemmatizzato
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text

### Seleziono i 20 pazienti casuali dal NOTEEVENTS + 1 esempio

In [82]:
subject_ids = []
distinct_subjects = noteevents_db.aggregate([
    {'$match': {'SUBJECT_ID': {'$gte': 0, '$lte': 10000}}},
    {'$group': {'_id': '$SUBJECT_ID'}},
    {'$limit': 5}
])

# Aggiungi i SUBJECT_ID distinti all'array
for doc in distinct_subjects:
    subject_ids.append(doc['_id'])

distinct_subjects = noteevents_db.aggregate([
    {'$match': {'SUBJECT_ID': {'$gte': 10000, '$lte': 20000}}},
    {'$group': {'_id': '$SUBJECT_ID'}},
    {'$limit': 5}
])

# Aggiungi i SUBJECT_ID distinti all'array
for doc in distinct_subjects:
    subject_ids.append(doc['_id'])

distinct_subjects = noteevents_db.aggregate([
    {'$match': {'SUBJECT_ID': {'$gte': 20000, '$lte': 30000}}},
    {'$group': {'_id': '$SUBJECT_ID'}},
    {'$limit': 5}
])

# Aggiungi i SUBJECT_ID distinti all'array
for doc in distinct_subjects:
    subject_ids.append(doc['_id'])

distinct_subjects = noteevents_db.aggregate([
    {'$match': {'SUBJECT_ID': {'$gte': 30000, '$lte': 40000}}},
    {'$group': {'_id': '$SUBJECT_ID'}},
    {'$limit': 5}
])

# Aggiungi i SUBJECT_ID distinti all'array
for doc in distinct_subjects:
    subject_ids.append(doc['_id'])

# Perchè è il nostro esempio
subject_ids.append(27431)

# Stampa i SUBJECT_ID nell'array
print(subject_ids)

[3497, 1590, 303, 1197, 1459, 18089, 16651, 14432, 17960, 11448, 22353, 21323, 20426, 25203, 23117, 30505, 30646, 30155, 31044, 30822, 27431]


In [ ]:
"""
[3497, 1590, 303, 1197, 1459, 18089, 16651, 14432, 17960, 11448, 22353, 21323, 20426, 25203, 23117, 30505, 30646, 30155, 31044, 30822, 27431]
"""

In [19]:
# Lista di id soggetto da considerare
query = list(noteevents_db.aggregate([
    {"$match": {"SUBJECT_ID": {"$in" : subject_ids}}},
    {"$project": {"_id": 0, "ROW_ID": 1, "SUBJECT_ID": 1, "CHARTDATE": 1, "GENDER": 1, "DOB": 1, "DOD": 1, "EXPIRE_FLAG": 1, "TEXT": 1}}
]))

df = pd.DataFrame(query)
print(df.shape)
df.head(10)

(51, 8)


,ROW_ID,SUBJECT_ID,CHARTDATE,TEXT,GENDER,DOB,DOD,EXPIRE_FLAG
0,722,27431,2148-03-07,Admission Date: [**2148-3-2**] D...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1
1,723,27431,2148-03-29,Admission Date: [**2148-3-20**] ...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1
2,724,27431,2148-04-08,Admission Date: [**2148-4-8**] D...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1
3,1907,21323,2139-05-04,Admission Date: [**2139-4-26**] ...,M,2082-12-21 00:00:00,None,0
4,1905,21323,2135-08-06,Admission Date: [**2135-7-30**] Discharge...,M,2082-12-21 00:00:00,None,0
5,1906,21323,2138-05-06,Admission Date: [**2138-4-21**] Dischar...,M,2082-12-21 00:00:00,None,0
6,5956,30155,2180-02-21,Admission Date: [**2180-2-21**] ...,F,2113-10-23 00:00:00,2180-02-21 00:00:00,1
7,5955,30155,2173-02-06,Admission Date: [**2173-2-3**] Discharg...,F,2113-10-23 00:00:00,2180-02-21 00:00:00,1
8,7321,20426,2155-07-13,Admission Date: [**2155-7-10**] Discharge...,F,2091-05-12 00:00:00,2161-09-11 00:00:00,1
9,7322,20426,2161-04-06,Admission Date: [**2161-4-1**] D...,F,2091-05-12 00:00:00,2161-09-11 00:00:00,1


## Section Extraction

### Effettuo il conteggio dei token prima della fase di estrazione delle sezioni

In [20]:
# Calcola la lunghezza dei caratteri per ciascuna riga nella colonna "TEXT"
df['TOKEN_COUNT'] = df['TEXT'].str.len() / 4

In [21]:
df.head(10)

,ROW_ID,SUBJECT_ID,CHARTDATE,TEXT,GENDER,DOB,DOD,EXPIRE_FLAG,TOKEN_COUNT
0,722,27431,2148-03-07,Admission Date: [**2148-3-2**] D...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1,3006.00
1,723,27431,2148-03-29,Admission Date: [**2148-3-20**] ...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1,4716.50
2,724,27431,2148-04-08,Admission Date: [**2148-4-8**] D...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1,709.75
3,1907,21323,2139-05-04,Admission Date: [**2139-4-26**] ...,M,2082-12-21 00:00:00,None,0,1039.25
4,1905,21323,2135-08-06,Admission Date: [**2135-7-30**] Discharge...,M,2082-12-21 00:00:00,None,0,1258.25
5,1906,21323,2138-05-06,Admission Date: [**2138-4-21**] Dischar...,M,2082-12-21 00:00:00,None,0,2498.00
6,5956,30155,2180-02-21,Admission Date: [**2180-2-21**] ...,F,2113-10-23 00:00:00,2180-02-21 00:00:00,1,2516.50
7,5955,30155,2173-02-06,Admission Date: [**2173-2-3**] Discharg...,F,2113-10-23 00:00:00,2180-02-21 00:00:00,1,1140.25
8,7321,20426,2155-07-13,Admission Date: [**2155-7-10**] Discharge...,F,2091-05-12 00:00:00,2161-09-11 00:00:00,1,1055.75
9,7322,20426,2161-04-06,Admission Date: [**2161-4-1**] D...,F,2091-05-12 00:00:00,2161-09-11 00:00:00,1,2278.50


### Carico il modello di MedSPACY per l'estrazione delle sezioni dai DISCHARGE SUMMARY

In [22]:
nlp = medspacy.load()

In [23]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

In [24]:
sectionizer = nlp.add_pipe("medspacy_sectionizer")

In [25]:
nlp.pipe_names

['medspacy_pyrush',
 'medspacy_target_matcher',
 'medspacy_context',
 'medspacy_sectionizer']

In [26]:
def section_extraction(text):
    doc = nlp(text)
    i = 0
    note_prep = ""
    for title in doc._.section_titles:
        if str(title).lower() == 'chief complaint:':
            note_prep += str(doc._.section_spans[i])
        if str(title).lower() == 'history of present illness:' or str(title).lower() == 'history:':
            note_prep += str(doc._.section_spans[i])
        if str(title).lower() == 'past medical history:':
            note_prep += str(doc._.section_spans[i])
        if str(title).lower() == 'discharge medications:':
            note_prep += str(doc._.section_spans[i])
        if str(title).lower() == 'brief hospital course:' or str(title).lower() == 'hospital course:':
            note_prep += str(doc._.section_spans[i])
        if str(title).lower() == 'discharge diagnoses:':
            note_prep += str(doc._.section_spans[i])
        i += 1

    return note_prep

In [27]:
# Creazione di una nuova colonna per il testo pulito
df['CLEANED TEXT'] = ''

# Iterazione sulle righe del DataFrame
for index, row in df.iterrows():
    # Ottenimento del testo dalla colonna desiderata (es. 'Text')
    text = row['TEXT']
    
    # Estrazione delle sezioni dal testo
    cleaned_text = section_extraction(text)
    
    # Salvataggio del testo pulito nella nuova colonna 'Cleaned Text'
    df.at[index, 'CLEANED TEXT'] = cleaned_text

### Effettuo la pulizia delle note cliniche

In [28]:
# Creazione di una nuova colonna per il testo lemmatizzato
df['LEMMATIZED TEXT'] = ''

# Iterazione sulle righe del DataFrame
for index, row in df.iterrows():
    # Ottenimento del testo dalla colonna desiderata (es. 'Text')
    text = row['CLEANED TEXT']
    
    # Tokenizzazione e lemmatizzazione del testo
    lemmatized_text = tokenize_and_lemmatize(text)
    
    # Salvataggio del testo lemmatizzato nella nuova colonna 'Lemmatized Text'
    df.at[index, 'LEMMATIZED TEXT'] = lemmatized_text

## Salvataggio del DataFrame con la colonna 'Lemmatized Text' nel file CSV

### Verifico il nuovo conteggio dei token dopo il preprocessing effettuato sulle note cliniche

In [29]:
# Calcola la lunghezza dei caratteri per ciascuna riga nella colonna "LEMMATIZED_TEXT"
df['TOKEN_COUNT_COMP'] = df['LEMMATIZED TEXT'].str.len() / 4

In [30]:
df = df.drop('TOKEN_COUNT', axis=1)
df = df.drop('CLEANED TEXT', axis=1)
df.head(10)

,ROW_ID,SUBJECT_ID,CHARTDATE,TEXT,GENDER,DOB,DOD,EXPIRE_FLAG,LEMMATIZED TEXT,TOKEN_COUNT_COMP
0,722,27431,2148-03-07,Admission Date: [**2148-3-2**] D...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1,Chief Complaint Hypotension/hypoxia Major Surg...,1005.25
1,723,27431,2148-03-29,Admission Date: [**2148-3-20**] ...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1,Chief Complaint Hypoxia Major Surgical Invasiv...,1676.50
2,724,27431,2148-04-08,Admission Date: [**2148-4-8**] D...,M,2070-10-04 00:00:00,2148-04-08 00:00:00,1,Chief Complaint hypoxia s/p PEA arrest Major S...,379.75
3,1907,21323,2139-05-04,Admission Date: [**2139-4-26**] ...,M,2082-12-21 00:00:00,None,0,Chief Complaint black stool Major Surgical Inv...,504.00
4,1905,21323,2135-08-06,Admission Date: [**2135-7-30**] Discharge...,M,2082-12-21 00:00:00,None,0,HISTORY PRESENT ILLNESS Patient 52-year-old ma...,561.25
5,1906,21323,2138-05-06,Admission Date: [**2138-4-21**] Dischar...,M,2082-12-21 00:00:00,None,0,HISTORY PRESENT ILLNESS Mr. Known firstname Kn...,350.00
6,5956,30155,2180-02-21,Admission Date: [**2180-2-21**] ...,F,2113-10-23 00:00:00,2180-02-21 00:00:00,1,Chief Complaint Called Emergency Department ev...,713.50
7,5955,30155,2173-02-06,Admission Date: [**2173-2-3**] Discharg...,F,2113-10-23 00:00:00,2180-02-21 00:00:00,1,CHIEF COMPLAINT Left facial pain trigeminal ne...,603.75
8,7321,20426,2155-07-13,Admission Date: [**2155-7-10**] Discharge...,F,2091-05-12 00:00:00,2161-09-11 00:00:00,1,HISTORY PRESENT ILLNESS patient 64-year-old wo...,619.50
9,7322,20426,2161-04-06,Admission Date: [**2161-4-1**] D...,F,2091-05-12 00:00:00,2161-09-11 00:00:00,1,Chief Complaint Melena Major Surgical Invasive...,446.00


### Salvo le note preprocessate

In [31]:
df.to_csv('preprocessed_noteevents.csv', sep=';', index=False)